In [9]:
import json

# METADATA_FILE = f'droid_raw/processed_0421/AUTOLab/success/*.json'

with open(METADATA_PATTERN) as f:
    f.load()

import glob

METADATA_PATTERN = 'droid_raw/1.0.1/AUTOLab/success/2023-07-07/Fri_Jul__7_09:42:23_2023/metadata_AUTOLab+*.json'

# 获取所有匹配的文件路径
metadata_files = glob.glob(METADATA_PATTERN)
metadata_files


FileNotFoundError: [Errno 2] No such file or directory: 'droid_raw/1.0.1/AUTOLab/success/2023-07-07/Fri_Jul__7_09:42:23_2023/metadata_AUTOLab+*.json'

In [21]:
import numpy as np
import json
import os
import trimesh
from scipy.spatial.transform import Rotation as R
import imageio.v2 as imageio
import glob
from PIL import Image
import glob
import h5py
import pyzed.sl as sl


date = "2023-11-25" # "2023-11-21" #"2023-11-24" #"2023-11-13" #"2023-09-05"#"2023-08-12"#"2023-10-27" #"2023-08-12" #"2023-11-30"
timestemp = "Sat_Nov_25_10:06:22_2023" #"Tue_Nov_21_11:23:03_2023" #"Tue_Nov_21_08:51:58_2023" #"Sat_Nov_25_10:06:22_2023" #"Fri_Nov_17_20:30:02_2023" #"Fri_Nov_24_18:42:53_2023" #"Mon_Nov_13_15:55:29_2023" #"Tue_Sep__5_08:50:21_2023" #"Fri_Jul__7_14:58:30_2023" #"Sat_Aug_12_12:15:52_2023"#"Fri_Oct_27_19:49:00_2023" # #"Thu_Nov_30_07:37:44_2023" #"Fri_Jul__7_09:44:34_2023" #"Fri_Jul__7_09:55:14_2023" #"Fri_Jul__7_09:44:34_2023" #"Fri_Jul__7_09:43:39_2023" #"Fri_Jul__7_09:42:23_2023"
camera = "22008760"#"24400334" #"22008760" #"24400334"
left_or_right = "right"
file_path = f'droid_raw/1.0.1/AUTOLab/success/{date}/{timestemp}/trajectory.h5'
index = 0



# --- CONFIG ---
CAM_NAMES = ['ext1', 'ext2']

# f"droid_raw/processed_0421/AUTOLab/success/{date}/{timestemp}/frames/{camera_alias}_depth_{index}.npz"
DEPTH_FILES = [
    f"droid_raw/processed_0421/AUTOLab/success/{date}/{timestemp}/frames/{camera_alias}_depth_{index}.npz" 
    for camera_alias in CAM_NAMES
    # 'data/droid_sample/ext2_depth_0.npz',
]

# METADATA_FILE = 'droid_raw/processed_0421/AUTOLab/success/metadata_AUTOLab+*.json'
INTRINSICS_FILE = 'data/droid_sample/intrinsics.json'
CAM2CAM_FILE = 'data/droid_sample/cam2cam.json'


RGB_FRAMES_DIR = f"droid_raw/processed_0421/AUTOLab/success/{date}/{timestemp}/frames/"#'data/droid_sample/png_frames'

METADATA_PATTERN = f'droid_raw/1.0.1/AUTOLab/success/{date}/{timestemp}/metadata_AUTOLab+*.json'
# 获取所有匹配的文件路径
METADATA_FILE = glob.glob(METADATA_PATTERN)[0]
# METADATA_FILE



# --- UTILS ---
def load_depth(path):
    d = np.load(path)
    # Try common keys
    for k in ['depth', 'arr_0', 'd', 'D']:
        if k in d:
            return d[k]
    # fallback: first array
    return list(d.values())[0]

def load_rgb_image(cam_serial, frame_idx=0):
    """Load RGB image for given camera serial and frame index"""
    # Look for RGB images matching the camera serial
    camera_alias = cam_serial.replace("22008760", "ext1").replace("24400334", "ext2")
    pattern = os.path.join(RGB_FRAMES_DIR, f"{camera_alias}_rgb_{index}.png")#f"{cam_serial}_*.png")
    rgb_files = sorted(glob.glob(pattern))
    
    # Use the specified frame index, or the first available frame
    if frame_idx < len(rgb_files):
        rgb_path = rgb_files[frame_idx]
    else:
        rgb_path = rgb_files[0]
    rgb_img = imageio.imread(rgb_path)
    # imageio loads as RGB by default, no need for color conversion
    return rgb_img

def crop_left_and_resize(rgb_img, depth_shape):
    """
    Crop left half of RGB image and resize to match depth shape
    rgb_img: RGB image with shape (H, 2*W, 3)
    depth_shape: (H_depth, W_depth) - target shape
    """
    h_rgb, w_rgb, _ = rgb_img.shape
    left_img = rgb_img[:, :w_rgb//2, :]
    h_depth, w_depth = depth_shape
    pil_img = Image.fromarray(left_img)
    resized_pil = pil_img.resize((w_depth, h_depth), Image.LANCZOS)
    resized_img = np.array(resized_pil)
    return resized_img

def load_intrinsics(intrinsics_file, uuid, cam_serial):
    """Load camera intrinsics from intrinsics.json file"""
    with open(intrinsics_file, 'r') as f:
        intrinsics_data = json.load(f)
    
    if uuid not in intrinsics_data:
        raise ValueError(f"UUID {uuid} not found in intrinsics file")
    
    if cam_serial not in intrinsics_data[uuid]:
        raise ValueError(f"Camera serial {cam_serial} not found for UUID {uuid}")
    
    cam_data = intrinsics_data[uuid][cam_serial]
    camera_matrix = cam_data['cameraMatrix']
    
    # Convert from [fx, cx, fy, cy] format to 3x3 matrix
    fx, cx, fy, cy = camera_matrix
    K = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0,  0,  1],
    ])
    return K

def transform_intrinsics_for_cropped_resized_image(K, original_width, original_height, target_width, target_height):
    K_new = K.copy()    
    scale_x = target_width / original_width
    scale_y = target_height / original_height
    K_new[0, 0] *= scale_x
    K_new[1, 1] *= scale_y
    K_new[0, 2] *= scale_x
    K_new[1, 2] *= scale_y
    
    return K_new

def load_cam2cam_transform(cam2cam_file, cam_name):
    """
    Load camera transformation from cam2cam.json
    cam_name: 'ext1' (left_cam) or 'ext2' (right_cam)
    Returns 4x4 transformation matrix
    """
    with open(cam2cam_file, 'r') as f:
        cam2cam_data = json.load(f)
    
    # Get the first (and likely only) entry
    uuid_key = list(cam2cam_data.keys())[0]
    data = cam2cam_data[uuid_key]
    
    if cam_name == 'ext1':
        # ext1 is left camera - assume identity transform
        pose_matrix = np.array(data['left_cam']['pose'])
        return pose_matrix
    elif cam_name == 'ext2':
        # ext2 is right camera - use its pose from cam2cam
        pose_matrix = np.array(data['right_cam']['pose'])
        return pose_matrix
    else:
        raise ValueError(f"Unknown camera name: {cam_name}")

def depth_to_points_with_colors(depth, rgb_img, K):
    """Convert depth to 3D points with RGB colors"""
    h, w = depth.shape
    i, j = np.meshgrid(np.arange(w), np.arange(h))
    
    # Filter out invalid depth values early
    valid_mask = np.isfinite(depth) & (depth > 100) & (depth < 10000)
    
    z = depth[valid_mask] / 1000.0
    print(f"Z range: {z.min()}, {z.max()}")
    i_valid = i[valid_mask]
    j_valid = j[valid_mask]
    
    x = (i_valid - K[0,2]) * z / K[0,0]
    y = (j_valid - K[1,2]) * z / K[1,1]
    pts = np.stack([x, y, z], axis=-1)
    
    # Extract colors for valid points
    colors = rgb_img[j_valid, i_valid]  # Note: j for rows, i for columns
    
    return pts, colors


def to_homogeneous_transform(pose_6d):
    """
    将一个 [x, y, z, rx, ry, rz] 格式的 6-DoF 位姿转换为 4x4 齐次变换矩阵
    假设旋转是轴角格式（Axis-angle）
    """
    translation = pose_6d[:3]
    rot_vector = pose_6d[3:]

    # 使用轴角构造旋转对象
    # rotation = R.from_rotvec(rot_vector)

    rotation = R.from_euler("xyz", np.array(rot_vector))#.as_matrix()

    # 构造 4x4 变换矩阵
    transform = np.eye(4)
    transform[:3, :3] = rotation.as_matrix()
    transform[:3, 3] = translation

    return transform



def get_intrinsic_parameters(svo_filepath, left_or_right = "left"):
    # 创建 ZED 相机对象
    zed = sl.Camera()

    # 设置初始化参数
    init_params = sl.InitParameters()
    init_params.set_from_svo_file(svo_filepath)
    init_params.svo_real_time_mode = False  # 不实时播放

    # 打开相机
    err = zed.open(init_params)
    if err != sl.ERROR_CODE.SUCCESS:
        # print("打开 SVO 文件失败: ", err)
        return

    # 获取相机信息
    camera_info = zed.get_camera_information()
    
    # print("\n=== 左摄像头内参 ===")
    left_cam_params = camera_info.camera_configuration.calibration_parameters.left_cam
    # print(f"分辨率: {left_cam_params.image_size}")
    resolution = left_cam_params.image_size
    print(f"分辨率: {resolution.width} x {resolution.height}")
    print(f"焦距 (fx, fy): {left_cam_params.fx}, {left_cam_params.fy}")
    print(f"光心 (cx, cy): {left_cam_params.cx}, {left_cam_params.cy}")
    # print(f"畸变系数: {list(left_cam_params.distortion_parameters)}")

    # print("\n=== 右摄像头内参 ===")
    right_cam_params = camera_info.camera_configuration.calibration_parameters.right_cam
    # print(f"分辨率: {right_cam_params.image_size}")
    resolution = left_cam_params.image_size
    print(f"分辨率: {resolution.width} x {resolution.height}")
    print(f"焦距 (fx, fy): {right_cam_params.fx}, {right_cam_params.fy}")
    print(f"光心 (cx, cy): {right_cam_params.cx}, {right_cam_params.cy}")

    zed.close()
    # return left_cam_params, right_cam_params

    if left_or_right == "left":
        cam_params = left_cam_params
    else:
        cam_params = right_cam_params

    fx, fy = cam_params.fx, cam_params.fy
    cx, cy = cam_params.cx, cam_params.cy


    fx /= 2
    fy /= 2
    cx /= 2
    cy /= 2
    # Convert from [fx, cx, fy, cy] format to 3x3 matrix
    # fx, cx, fy, cy = camera_matrix
    K = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0,  0,  1],
    ])
    return K


def main():
    with open(METADATA_FILE, 'r') as f:
        meta = json.load(f)
    
    uuid = meta['uuid']
    print(f"Processing UUID: {uuid}")
    
    # Camera serial mapping
    cam_serials = {
        'ext1': meta['ext1_cam_serial'],
        'ext2': meta['ext2_cam_serial'],
        'wrist': meta['wrist_cam_serial']
    }
    
    for cam, depth_file in zip(CAM_NAMES, DEPTH_FILES):
        print(f'\n=== Processing {cam} ===')
        
        # Check if depth file exists
        if not os.path.exists(depth_file):
            print(f'Warning: Depth file {depth_file} not found, skipping {cam}')
            continue
            
        depth = load_depth(depth_file)
        if depth.ndim == 3:
            depth = depth[0]  # take first frame if needed
        h, w = depth.shape
        print(f"Depth shape: {depth.shape}, range: [{np.nanmin(depth):.3f}, {np.nanmax(depth):.3f}]")
        
        # Load RGB image
        cam_serial = cam_serials[cam]
        rgb_img = load_rgb_image(cam_serial)
        if rgb_img is None:
            print(f'Warning: Could not load RGB image for {cam}, skipping camera')
            continue
        
        # Crop left half and resize to match depth
        rgb_img = crop_left_and_resize(rgb_img, depth.shape)
        
        # Load actual camera intrinsics
        # K = load_intrinsics(INTRINSICS_FILE, uuid, cam_serial)
        svo_filepath = f"droid_raw/1.0.1/AUTOLab/success/{date}/{timestemp}/recordings/SVO/{cam_serial}.svo"
        K = get_intrinsic_parameters(svo_filepath, left_or_right = "left")
        
        # # Get original image dimensions from intrinsics file
        # with open(INTRINSICS_FILE, 'r') as f:
        #     intrinsics_data = json.load(f)
        # original_width = intrinsics_data[uuid][cam_serial]['width']
        # original_height = intrinsics_data[uuid][cam_serial]['height']
        
        # Transform K to account for cropping left half and resizing to depth dimensions
        # target_height, target_width = depth.shape
        # K_transformed = transform_intrinsics_for_cropped_resized_image(
        #     K, original_width, original_height, target_width, target_height
        # )
        
        # print(f"Original intrinsics: {K}")
        # print(f"Original image size: {original_width}x{original_height}")
        # print(f"Target image size: {target_width}x{target_height}")
        # print(f"Transformed intrinsics: {K_transformed}")
        # print(f"Depth shape: {depth.shape}")
        # print(f"RGB shape: {rgb_img.shape}")
        
        # Load camera transformation from cam2cam
        # T_cam2world = load_cam2cam_transform(CAM2CAM_FILE, cam)

        with h5py.File(file_path, 'r') as f:
            # 获取 action/cartesian_position 数据
            # cartesian_pos = f['action/cartesian_position'][()]
            camera_extrinsic = f[f'observation/camera_extrinsics/{cam_serial}_{left_or_right}'][()][index]

        T_cam2world = to_homogeneous_transform(camera_extrinsic)
        
        # Convert depth to 3D points in camera frame with colors
        K_transformed = K
        pts_cam, colors = depth_to_points_with_colors(depth, rgb_img, K_transformed)
        
        if len(pts_cam) == 0:
            print(f'No valid points found for {cam}')
            continue
        
        print(f"Points in camera frame: {pts_cam.shape[0]}")
        
        # Transform points from camera frame to world frame using cam2cam matrix
        # Convert to homogeneous coordinates
        pts_h = np.concatenate([pts_cam, np.ones((pts_cam.shape[0],1))], axis=1)
        
        # Apply cam2world transformation: pts_world = T_cam2world @ pts_cam
        pts_world = (T_cam2world @ pts_h.T).T[:, :3]
        
        # Create colored trimesh point cloud
        out_path = f"output/{date}/{timestemp}/pointcloud/{cam}_pointcloud_{index}_haoyu.ply"
        # f"output/{date}/{timestemp}/pointcloud/combined_pointcloud_{index}_haoyu.ply"#f'data/droid_sample/{cam}_colored_cloud.ply'
        colors = colors.astype(np.float32) / 255.0
        print(f"Colors shape: {colors.shape}")
        print(f"Points shape: {pts_world.shape}")
        point_cloud = trimesh.PointCloud(vertices=pts_world, colors=colors)
        point_cloud.export(out_path)
        print(f'✓ Saved colored point cloud: {out_path} ({pts_world.shape[0]} points)')

if __name__ == '__main__':
    main()

    

Processing UUID: AUTOLab+44bb9c36+2023-11-25-10h-06m-22s

=== Processing ext1 ===
Depth shape: (360, 640), range: [218.324, inf]
[2025-07-16 14:12:46 UTC][ZED][INFO] Logging level INFO
分辨率: 1280 x 720
焦距 (fx, fy): 524.3548583984375, 524.3548583984375
光心 (cx, cy): 639.7761840820312, 370.2777099609375
分辨率: 1280 x 720
焦距 (fx, fy): 524.3548583984375, 524.3548583984375
光心 (cx, cy): 639.7761840820312, 370.2777099609375
[2025-07-16 14:12:46 UTC][ZED][INFO] [Init]  Depth mode: NEURAL
[2025-07-16 14:12:46 UTC][ZED][INFO] [Init]  Serial Number: S/N 22008760
Z range: 0.2183241993188858, 9.996537208557129
Points in camera frame: 160813
Colors shape: (160813, 4)
Points shape: (160813, 3)
✓ Saved colored point cloud: output/2023-11-25/Sat_Nov_25_10:06:22_2023/pointcloud/ext1_pointcloud_0_haoyu.ply (160813 points)

=== Processing ext2 ===
Depth shape: (360, 640), range: [-inf, inf]
[2025-07-16 14:12:46 UTC][ZED][INFO] Logging level INFO
分辨率: 1280 x 720
焦距 (fx, fy): 531.6160888671875, 531.616088867187